<a href="https://colab.research.google.com/github/DenisKuts1/dataset/blob/master/Web_mining_%7C_543_18_%D0%9A%D1%83%D1%86_%D0%94%D0%B5%D0%BD%D0%B8%D1%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 1

In [0]:
import pandas as pd

url = "https://raw.githubusercontent.com/DenisKuts1/dataset/master/ign.csv"
dataFrame = pd.read_csv(url, error_bad_lines=False)  
dataFrame.head()

,Unnamed: 0,score_phrase,title,url,platform,score,genre,editors_choice,release_year,release_month,release_day
0,0,Amazing,LittleBigPlanet PS Vita,/games/littlebigplanet-vita/vita-98907,PlayStation Vita,9.0,Platformer,Y,2012,9,12
1,1,Amazing,LittleBigPlanet PS Vita -- Marvel Super Hero E...,/games/littlebigplanet-ps-vita-marvel-super-he...,PlayStation Vita,9.0,Platformer,Y,2012,9,12
2,2,Great,Splice: Tree of Life,/games/splice/ipad-141070,iPad,8.5,Puzzle,N,2012,9,12
3,3,Great,NHL 13,/games/nhl-13/xbox-360-128182,Xbox 360,8.5,Sports,N,2012,9,11
4,4,Great,NHL 13,/games/nhl-13/ps3-128181,PlayStation 3,8.5,Sports,N,2012,9,11


In [0]:
dataFrame["score"].describe()

count    18625.000000
mean         6.950459
std          1.711736
min          0.500000
25%          6.000000
50%          7.300000
75%          8.200000
max         10.000000
Name: score, dtype: float64

In [0]:
dataFrame["score"].mode()

0    8.0
dtype: float64

In [0]:
dataFrame["score"].median()

7.3

In [0]:
dataFrame["score"].mean()

6.950459060402666

# Lab2

*Варіант для парсингу* - рейтинг відеоігор [Metacritic](https://www.metacritic.com/).

*Запитання до ігор*: 
- Найкращі 10 ігор за жанром.
- Розробник з найбільшою кількістю ігор.
- Найбільш та найменш оцінені експертами ігри заданим розробником.
- Найбільш та найменш оцінені користувачами ігри заданим розробником.
- Ігри заданого розробника з найбільшою різницею між рейтингами користувачів і експертів.
- Найкращі розробники ігор за заданим жанром.

In [0]:
try:
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess
import json
import logging


In [0]:
from enum import Enum

# enum that helps piplines to differentiate between game and publisher items 
class ItemType(Enum):
    GAME = 1
    PUBLISHER = 2

In [0]:
# pipeline that handles game item and saves it to games.jl file
class GamesWriterPipeline(object):
    
    game_id = 0
  
    def open_spider(self, spider):
        self.file = open('games.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        if item["type"] is ItemType.GAME:
            game_item = {
                'id': self.game_id,
                'title': item["title"],
                'metascore': item["metascore"],
                'user_score': item["user_score"],
                'rating': item["rating"],
                'ganre': item["ganre"],
                'developer': item["developer"],
                'publisher': item["publisher"],
                'release_date': item["release_date"],
            }
            self.game_id += 1
            line = json.dumps(dict(game_item)) + "\n"
            self.file.write(line)
        return item

In [0]:
# pipeline that handles publisher item and saves it to publishers.jl file
class PublisherJsonWriterPipeline(object):
  
    publisher_id = 0
    
    def open_spider(self, spider):
        self.file = open('publishers.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        #if item is not None:
          if item['type'] is ItemType.PUBLISHER:
            publisher_item = {
                'id': self.publisher_id,
                'title': item["title"],
                'average_score': item["average_score"],
                'highest_score': item["highest_score"],
                'highest_score_game': item["highest_score_game"],
                'lowest_score': item["lowest_score"],
                'lowest_score_game': item["lowest_score_game"],
            }
            self.publisher_id += 1
            line = json.dumps(dict(publisher_item)) + "\n"
            self.file.write(line)
          return item

In [0]:
# set of visited publisher pages, needed for preventing publisher duplications
publisher_links = set()

# implementation of scrapy spider
class MetacriticSpider(scrapy.Spider):
    name = "metacritic"
    start_urls = [
        #'https://www.metacritic.com/browse/games/score/metascore/all/pc/filtered?sort=desc' #takes way too long to process
        'https://www.metacritic.com/browse/games/score/metascore/90day/pc/filtered?sort=desc', #shorter version, consists only of games that have been published during last 90 days
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {
            '__main__.GamesWriterPipeline': 1,
            '__main__.PublisherJsonWriterPipeline': 2
        },

    }
    
    
    # function that parses main link and all of next pages for links to games
    def parse(self, response):
        body = response.xpath('//div[@id="main"]')
        for quote in body.css('div.product_wrap'):
            # handling a link to a game page
            game_page = quote.css('a::attr(href)').get()
            game_page = response.urljoin(game_page)
            yield scrapy.Request(game_page, callback=self.parse_game)
            
        # checking if there are any additional pages in the list
        next_page = response.css('span.flipper.next').css('a::attr(href)').get()
        if next_page is not None:
            # handling the next page
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)
            
            
    # function that parses game page
    def parse_game(self, response):
        body = response.xpath('//div[@id="main"]')
        yield {
                'type': ItemType.GAME,
                'title': body.css('div.product_title h1::text').extract_first(),
                'release_date': body.css('li.summary_detail.release_data span.data::text').extract_first(),
                'metascore': body.css('div.metascore_w.xlarge.game span::text').extract_first(),
                'user_score': body.css('div.metascore_w.user.large.game::text').extract_first(),
                'rating': body.css('li.summary_detail.product_rating span.data::text').extract_first(),
                'ganre': body.css('li.summary_detail.product_genre span.data::text').extract(),
                'developer': body.css('li.summary_detail.developer span.data::text').extract_first(),
                'publisher': body.css('li.summary_detail.publisher span.data a::text').extract_first(),
            }
        # finding out have the publisher been parsed already
        publisher_page = body.css('li.summary_detail.publisher span.data a::attr(href)').get()
        if not publisher_page in publisher_links:
            # handling a link to a publisher page
            publisher_links.add(publisher_page)
            publisher_page = response.urljoin(publisher_page)
            yield scrapy.Request(publisher_page, callback=self.parse_publisher)
        
    
    #function that parses publisher page
    def parse_publisher(self, response):
        body = response.xpath('//div[@id="main"]')
        yield {
                'type': ItemType.PUBLISHER,
                'title': body.css('h1.person_title::text').extract_first(),
                'average_score': body.css('tr.review_average span.data.textscore::text').extract_first(),
                'highest_score': body.css('tr.highest_review span.metascore_w.small.game::text').extract_first(),
                'highest_score_game': body.css('tr.highest_review span.product_title a::text').extract_first(),
                'lowest_score': body.css('tr.lowest_review span.metascore_w.small.game::text').extract_first(),
                'lowest_score_game': body.css('tr.lowest_review span.product_title a::text').extract_first(),
            }

        

In [7]:
# carrying out crawler process
# works only once, if you want to repeat it, you should restart the runtime
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
})

process.crawl(MetacriticSpider)
process.start()

2019-05-09 17:30:27 [scrapy.utils.log] INFO: Scrapy 1.6.0 started (bot: scrapybot)
2019-05-09 17:30:27 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 19.2.0, Python 3.6.7 (default, Oct 22 2018, 11:32:17) - [GCC 8.2.0], pyOpenSSL 19.0.0 (OpenSSL 1.1.1b  26 Feb 2019), cryptography 2.6.1, Platform Linux-4.14.79+-x86_64-with-Ubuntu-18.04-bionic
2019-05-09 17:30:27 [scrapy.crawler] INFO: Overridden settings: {'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}


In [0]:
import pandas as pd

# creating data frames from json files

games_data_frame = pd.read_json('games.jl', lines = True)
games_data_frame.set_index('id', inplace=True)

publisher_data_frame = pd.read_json('publishers.jl', lines = True)
publisher_data_frame.set_index('id', inplace=True)

In [12]:
#visualizing first 5 games from games data frame
games_data_frame.head()

,developer,ganre,metascore,publisher,rating,release_date,title,user_score
id,,,,,,,,
0,\n 3DClouds.it,"[Racing, Arcade, Automobile]",64,\n ...,E,"Mar 26, 2019",Xenon Racer,6.4
1,\n From Software ...,"[Action Adventure, General]",89,\n ...,M,"Mar 22, 2019",Sekiro: Shadows Die Twice,7.8
2,\n Storm in a Teacup ...,"[Adventure, 3D, First-Person]",65,\n ...,M,"May 2, 2019",Close to the Sun,6.2
3,\n Codemasters,"[Racing, Simulation, Automobile]",85,\n ...,E,"Feb 26, 2019",DiRT Rally 2.0,5.1
4,"\n btf GmbH, btf ...","[Adventure, General]",67,\n ...,None,"Mar 12, 2019",Truberbrook: A Nerd Saves the World,4.8


In [13]:
#visualizing first 5 publishers from publishers data frame
publisher_data_frame.head()

,average_score,highest_score,highest_score_game,lowest_score,lowest_score_game,title
id,,,,,,
0,86,87.0,Baba Is You,84.0,Baba Is You,Hempuli Oy
1,64,82.0,Victor Vran: Overkill Edition,41.0,Lets Fish! Hooked On,Wired Productions
2,68,98.0,Tony Hawk's Pro Skater 2,21.0,Fast & Furious: Showdown,Activision
3,65,70.0,Tech Support: Error Unknown,59.0,Arelite Core,Dragon Slumber
4,67,86.0,Bridge Constructor Portal,41.0,Super Blackjack Battle II Turbo Edition: The C...,Headup Games


# Lab3

In [22]:
dfjson["developer"] = dfjson["developer"].str.strip()
dfjson["publisher"] = dfjson["publisher"].str.strip()
dfjson["metascore"] = dfjson["metascore"].astype(int)
dfjson["user_score"] = pd.to_numeric(dfjson["user_score"], errors='coerce')  
dfjson

,developer,ganre,metascore,publisher,rating,title,user_score
0,Saber Interactive,"[Action, Shooter, First-Person, Tactical]",72,Saber Interactive,M,World War Z,6.0
1,Clifftop Games,"[Adventure, Point-and-Click]",77,Raw Fury,None,Whispers of a Machine,8.0
2,Madruga Works,"[Strategy, Real-Time, General]",74,Madruga Works,None,Dawn of Man,6.9
3,Owlchemy Labs,"[Action, General]",77,Owlchemy Labs,None,Vacation Simulator,NaN
4,inkle,"[Adventure, General]",76,inkle,None,Heaven's Vault,5.4
5,Eastshade Studios,"[First-Person, Adventure, 3D]",78,Eastshade Studios,None,Eastshade,8.1
6,Polyslash,"[Adventure, General]",77,Klabater,None,We. The Revolution,8.2
7,From Software,"[Action Adventure, General]",89,Activision,M,Sekiro: Shadows Die Twice,7.8
8,Ubisoft Montreal,"[Action, Shooter, First-Person, Arcade]",73,Ubisoft,M,Far Cry New Dawn,3.5
9,Capcom,"[Action Adventure, General]",89,Capcom,M,Devil May Cry 5,8.7
